## Newsletter Text-To-Speech - Kokoro

### Import Modules

In [ ]:
# 🧠 2. Import modules
import soundfile as sf
from IPython.display import display, Audio
import tempfile, shutil, subprocess, os
import numpy as np
from pathlib import Path
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
from kokoro import KPipeline
import torch

from huggingface_hub import login
import config
if config.ACCESS_TOKEN:
    login(token=config.ACCESS_TOKEN)

### Helper Functions

In [ ]:
# 🎙️ 3. Helper functions
def combine_audio_files(file_list, output_path):
    """Combine WAV files by re-encoding each and concatenating via numpy + soundfile."""
    combined = []
    for file in file_list:
        data, sr = sf.read(file)
        if sr != 24000:
            raise ValueError(f"Unexpected sample rate: {sr} in {file}")
        combined.append(data)

    all_audio = np.concatenate(combined)
    sf.write(output_path, all_audio, 24000)

def generate_speech(text, title="output", voice="af_heart", split_pattern=r"\n+", speed=1.0, lang_code="a"):
    """Generate complete audio for given text using Kokoro."""
    # 🇺🇸 'a' => American English, 🇬🇧 'b' => British English
    pipeline = KPipeline(lang_code=lang_code, repo_id="hexgrad/Kokoro-82M")
    audio_parts = []

    with tempfile.TemporaryDirectory() as temp_dir:
        temp_dir_path = Path(temp_dir)

        generator = pipeline(
            text, voice=voice, speed=speed, split_pattern=split_pattern
        )

        for i, (gs, ps, audio) in enumerate(generator):
            if audio is None or len(audio) == 0:
                continue
            part_path = temp_dir_path / f"part_{i}.wav"
            sf.write(str(part_path), audio, 24000)
            audio_parts.append(str(part_path))

        if not audio_parts:
            return None

        final_output = f"{title}.wav"
        if len(audio_parts) == 1:
            shutil.copy2(audio_parts[0], final_output)
        else:
            combine_audio_files(audio_parts, final_output)

        return final_output

### Test Run

In [ ]:
title = "Test"
text = "Can you run this test please? I say, run it!"

In [ ]:
# 🌀 Generate the final audio
final_audio_path = generate_speech(text, title)

# ▶️ Play and download
if final_audio_path:
    display(Audio(final_audio_path, autoplay=True))
else:
    print("No audio generated.")

### Convert `.txt` Articles to Audio

In [ ]:
directory_path = Path('output/')

if directory_path.is_dir():
    for entry in directory_path.iterdir():
        with open(entry, "r", encoding="utf-8") as f:
            text = f.read()
        final_audio_path = generate_speech(text, title=entry.stem)